### What is ANOVA?


ANOVA (Analysis of Variance) is a statistical method(s) used to analyze the difference of **means** among the groups. 



It may seem similar to t-test, but it is more general and can be used to compare more than two groups. It is used to test the null hypothesis that there is no significant difference between the means of the groups. If the p-value is less than the significance level (usually 0.05), then we reject the null hypothesis and conclude that there are significant differences between the groups.

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from plotly import express, graph_objects, io
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison 
import patsy

io.templates.default = 'plotly_dark'

| **Model**                  | **Independent**         | **Dependent** |
| -------------------------- | ----------------------- | ------------- |
| Simple Linear Regression   | interval                | interval      |
| One-Way ANOVA              | nominal                 | interval      |
| One-Way ANCOVA             | nominal and interval    | interval      |
| Binary Logistic Regression | nominal and/or interval | binary        |

- Think of `interval` as continuous data and `nominal` as categorical data.

In [19]:
# data generation
np.random.seed(10)
treatments = np.repeat(['A', 'B', 'C'], 10) # repeat each treatment 10 times
depA = np.round(stats.norm.rvs(loc=100, scale=10, size=10), 1) # from normal dist
depB = np.round(stats.norm.rvs(loc=105, scale=7, size=10), 1)    
depC = np.round(stats.norm.rvs(loc=95, scale=5, size=10), 1)    

In [20]:
df = pd.DataFrame({
    'Treatment':treatments,
    'Dependent': np.array([depA, depB, depC]).flatten()
    })
df.Treatment = pd.Categorical(df.Treatment, categories=['A', 'B', 'C']) 
# overwriting the treatment column with a categorical type
display(df.head())

,Treatment,Dependent
0,A,113.3
1,A,107.2
2,A,84.5
3,A,99.9
4,A,106.2


### Exploratatory Analysis

In [21]:
# summary statistics
df.groupby('Treatment').Dependent.describe()

/var/folders/sp/gm0tt5zx5t9dh__5x5l8g1600000gn/T/ipykernel_45664/3006084385.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,count,mean,std,min,25%,50%,75%,max
Treatment,,,,,,,,
A,10.0,100.60,7.937674,84.5,98.700,100.55,105.325,113.3
B,10.0,106.22,6.713635,97.0,100.125,107.30,111.175,115.4
C,10.0,96.78,7.055778,85.1,94.075,97.20,101.650,106.9


In [22]:
express.box(df, x='Treatment', y='Dependent', points='all', title='Boxplot of Dependent by Treatment')

In [23]:
express.scatter(df, x='Treatment', y='Dependent', title='Scatterplot of Dependent by Treatment')

- Regular linear regression cannot be used since the target variable is not continuous. Another model must be used.
- ANOVA takes the mean of each of the groups and compares them to see if they are significantly different.

### Linear Model

- The linear model for ANOVA is similar to the linear regression model. The only difference is that the independent variables are categorical. 

$$ \hat{Y} = \hat{\beta}_0 + \hat{\beta}_1 B + \hat{\beta}_2 C + \varepsilon $$

- $B$ and $C$ are the categorical variables.
- $\hat{\beta}_0$ is the intercept.
- $\hat{\beta}_1$ and $\hat{\beta}_2$ are the coefficients for the categorical variables.

\begin{align*}
    A:& \quad \text{ dependent variable} = \hat{\beta}_0 \\
    B:& \quad \text{ dependent variable} = \hat{\beta}_0 + \hat{\beta}_1\\
    C:& \quad \text{ dependent variable} = \hat{\beta}_0 + \hat{\beta}_2\\
\end{align*}

Dummy variables demo

In [26]:
print(df.Treatment[:5]) # AAAAA -> dummy variables/one-hot encoding


0    A
1    A
2    A
3    A
4    A
Name: Treatment, dtype: category
Categories (3, object): ['A', 'B', 'C']


**Base Case: A**

| **Treatment** | A | B | C |
| ------------- | - | - | - |
| A             | 1 | 0 | 0 |
| B             | 0 | 1 | 0 |
| C             | 0 | 0 | 1 |

- The base case is the reference group. (i.e. compare B with respect to A what is the difference?)

This can be represented as while A is in your model, B and C are are being compared.


| **Treatment**  | B | C |
| ------------- | - | - |
| B             | 1 | 0 |
| C             | 0 | 1 |


### Assumptions (iid)

1. **Normality**: The residuals are normally distributed.
2. **Identically**: The variance of the residuals is constant.
3. **Independent Observations**: The observations are independent of each other.


### Hypothesis
- **Null**: $$ H_0 : \quad \hat{\beta}_1 = \hat{\beta}_2 = 0 $$
- **Alternative**: $$ H_1 : \quad \text{At least one of the } \hat{\beta}_i \neq 0 $$

### Model

In [27]:
linear_model = ols('Dependent ~ Treatment', data=df).fit() # ordinary least squares
linear_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Dependent   R-squared:                       0.241
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                     4.285
Date:                Tue, 11 Jun 2024   Prob (F-statistic):             0.0242
Time:                        22:05:34   Log-Likelihood:                -100.43
No. Observations:                  30   AIC:                             206.9
Df Residuals:                      27   BIC:                             211.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        100.6000      2.294     43.855      0.000      95.893     105.307
Treatment[T.B]     5.6200      3.244      1.732      0.095      -1.036      12.276
Treatment[T.C]    -3.8200      3.244     -1.178      0.249     -10.476       2.836
==============================================================================
Omnibus:                        1.137   Durbin-Watson:                   1.902
Prob(Omnibus):                  0.566   Jarque-Bera (JB):                1.079
Skew:                          -0.410   Prob(JB):                        0.583
Kurtosis:                       2.563   Cond. No.                         3.73
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
anova_table = anova_lm(linear_model)
print(anova_table) 

             df    sum_sq     mean_sq         F    PR(>F)
Treatment   2.0   450.968  225.484000  4.285042  0.024197
Residual   27.0  1420.772   52.621185       NaN       NaN


In [34]:
y,X = patsy.dmatrices('Dependent ~ Treatment', data=df)
X[:5] # show first 5 rows of the design matrix
y[:5] # show first 5 rows of the response vector

# to np.array
X,y = np.array(y),np.array(X)

Using Normal Equations:
$$\hat{\beta} = (X^T X )^{-1} X^T y$$

In [55]:
XT = X.T
XTX = np.matmul(XT,X)
XTX_inv = np.linalg.inv(XTX)
XTX_invXT = np.matmul(XTX_inv,XT)
beta = np.matmul(XTX_invXT,y).flatten()
beta

array([0.00982159, 0.00343626, 0.00313087])

In [50]:
beta0,beta1,beta2 = beta[0], beta[1], beta[2]

In [54]:
def research(level):
    if level == 'A':
        return beta0
    elif level == 'B':
        return beta0 + beta1
    else:
        return beta0 + beta2
    
# level A, B, C
research('A'), research('B'), research('C')

(0.0098215893414922, 0.013257851593973429, 0.012952463572288032)

In [59]:
# mean of dependent variable for treatment A
df.loc[df.Treatment == 'A', 'Dependent'].mean(), df.loc[df.Treatment == 'B', 'Dependent'].mean() ,df.loc[df.Treatment == 'C', 'Dependent'].mean()

(100.6, 106.22, 96.78)

In [60]:
linear_model.bse # standard errors

Intercept         2.293931
Treatment[T.B]    3.244108
Treatment[T.C]    3.244108
dtype: float64

In [62]:
se0,se1,se2 = linear_model.bse
linear_model.tvalues # t-values

Intercept         43.854854
Treatment[T.B]     1.732371
Treatment[T.C]    -1.177519
dtype: float64

ANOVA Table:
- df: degrees of freedom
- SS: sum of squares (errors/residuals)
- MS: mean squares (errors/residuals)
- F: F-statistic
- p-value: probability for the F-statistic

In [63]:
# 
anova_lm(linear_model)

,df,sum_sq,mean_sq,F,PR(>F)
Treatment,2.0,450.968,225.484000,4.285042,0.024197
Residual,27.0,1420.772,52.621185,NaN,NaN


In [65]:
linear_model.fittedvalues[:5] # first 5 fitted values

$$SSR = \sum_{i=1}^{N} (\hat{y}_i - \overline{y})^2 $$

- The `ess` attribute returns the sum of squares due to regression.

In [66]:
df['Estimate'] = linear_model.fittedvalues

In [72]:
linear_model.ess

450.96800000000053

In [73]:
ssr = np.sum((df.Estimate - df.Dependent.mean())**2)
ssr

450.9679999999983

$$SSE = \sum_{i=1}^{N} (\hat{y} - y_i)^2 $$

- The `ssr` attribute returns the sum of squares due to error.

In [74]:
# sum of squared residuals/sum of squared errors
linear_model.ssr

1420.7720000000002

In [81]:
N = df.shape[0] # number of observations
K = 3 # number of parameters
df1 = K-1 # = 2
df2 = N-K   # = 27 

linear_model.df_model # degrees of freedom for the model (K-1) = 2
linear_model.df_resid # degrees of freedom for the residuals (N-K) = 27

27.0

F Statistic compares the ratios of the mean squares.

$$F = \frac{MSR}{MSE}$$

In [87]:
mse = linear_model.mse_resid # mean squared error
msr = linear_model.mse_model # mean squared regression

In [90]:
linear_model.fvalue # F-statistic = 4.2850422164851265
f = msr/mse
f

4.2850422164851265

In [98]:
xvals = np.linspace(stats.f.ppf(0.01, df1, df2), stats.f.ppf(0.99, df1, df2), 100)
f_crit = stats.f.pdf(0.95, df1, df2)

fig = graph_objects.Figure()
fig.add_trace(graph_objects.Scatter(
    x=xvals, 
    y=stats.f.pdf(xvals, df1, df2), 
    mode='lines', 
    name='PDF'))
fig.add_trace(graph_objects.Scatter(
    x=[f_crit],
    y=[0, 1.0], 
    mode='markers', 
    marker={'color':'red', 'size':20},    
    name='Critical F', line=dict(dash='dash')
))
fig.add_trace(graph_objects.Scatter(
    x=[f],
    y=[0],
    mode='markers', 
    marker={'color':'green', 'size':20},
    name='F-value'  
))
fig.update_layout(title='F-distribution', xaxis_title='F-value', yaxis_title='Density')